Travail developpe par le binome :
**Bruno OLIVEIRA, Samuel GHEZI**

Sur orientation de le Professeur **Martin GHIENNE**.

**Introduction**

L’exploitation d’un avion s’accompagne de conditions de vol très variables et difficiles à prédire, ce qui rend complexe l’estimation précise des chargements structuraux rencontrés en situation réelle. Bien que des informations telles que les déformations et les contraintes soient essentielles pour optimiser la maintenance et améliorer les modèles de dimensionnement, ces grandeurs ne sont généralement pas mesurées directement sur les aéronefs commerciaux. L’installation de capteurs dédiés entraînerait en effet une augmentation significative des coûts, de la masse, de la complexité d’intégration et des exigences de certification.

Le Mini-Challenge propose ainsi de développer un capteur virtuel basé sur des méthodes d’apprentissage automatique, capable d’estimer l’état de contrainte structurelle en différents points de l’avion à partir des seuls paramètres déjà enregistrés par l’instrumentation de bord. L’objectif est de prédire des grandeurs non mesurées physiquement, mais inférées à partir de variables de vol telles que l’attitude, les vitesses, les accélérations, les ordres de commande et les conditions de vent.

Pour cela, un ensemble de données réelles provenant de 44 vols d’essai est mis à disposition. Ce jeu de données comprend :

39 paramètres issus de l’instrumentation de bord, représentant l’état de vol, les efforts aérodynamiques et les actions de contrôle ;

15 jauges d’extensométrie (en micro-déformations, με) positionnées en différents points structuraux de l’appareil, permettant de mesurer directement les contraintes locales.

En résumé, ce projet vise à démontrer la capacité d’un modèle d’apprentissage supervisé à reproduire les contraintes structurelles réelles à partir de données opérationnelles courantes, ouvrant la voie à des stratégies de maintenance plus prédictives, moins coûteuses et mieux informées, dans la continuité des travaux précédents sur les capteurs virtuels pour le suivi de santé structurale.

Pour cette projet, on va se baser dans la teorie de Machine Learning, en se basent dans l'image ci dessous :

**Management des données**

Le liste de variable avec les description et sont format qui existe dans les données collectées pendant les vols sont en dessous :

In [2]:
# Importing Lybraries
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from tqdm import tqdm
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.nn.init import kaiming_uniform_

**Management des données** - Acquisition des données

Displayng the dataframes

**Management des données** - Exploration des Données

Ploting grqphs for correlation

**Management des données** - Préparation des Données

In [4]:
df = pd.read_csv('df.csv')

In [5]:
df = df.drop(columns='Relative_Time')

In [4]:
df.head(5)

,Nz,Nx,Roll_Angle,Pitch_Angle,True_AOA,True_Sideslip,FPA,True_Heading,CAS,TAS,...,Strain7,Strain8,Strain9,Strain10,Strain11,Strain12,Strain13,Strain14,Strain15,Tol_ID
0,0.906212,-0.014912,0.147686,-0.311379,3.590025,-2.303673,146.080303,134.823861,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,TOL_1
1,0.906212,-0.014912,0.147686,-0.311379,3.590025,-2.303673,146.080303,134.823861,0.0,0.0,...,-0.601572,-0.884219,-0.463841,0.161175,-0.000325,-0.143096,0.403351,0.006286,0.175358,TOL_1
2,0.906212,-0.014912,0.147686,-0.311379,3.590025,-2.303673,146.080303,134.824664,0.0,0.0,...,-0.310433,-1.410038,-1.853289,0.646155,-0.000651,-0.571435,0.818918,0.012573,0.350715,TOL_1
3,0.906212,-0.014912,0.147686,-0.311379,3.590025,-2.303673,146.080303,134.825466,0.0,0.0,...,0.424748,-1.414549,-1.854326,0.174219,-0.000976,-1.013577,1.216162,-1.045466,-0.521183,TOL_1
4,0.906212,-0.014912,0.147686,-0.311379,3.590025,-2.303673,146.080303,134.825466,0.0,0.0,...,0.572497,-2.114138,2.512348,0.644700,0.579980,-0.572384,1.613406,0.691031,0.003260,TOL_1


In [6]:
# Targets = deformações (saídas do modelo)
target_cols = [f"Strain{i}" for i in range(1, 16)]

# Features = todas as colunas menos Strains e Flight_ID
feature_cols = [c for c in df.columns if c not in target_cols + ["Tol_ID"]]

print("Features:", feature_cols)
print("Targets :", target_cols)

Features: ['Nz', 'Nx', 'Roll_Angle', 'Pitch_Angle', 'True_AOA', 'True_Sideslip', 'FPA', 'True_Heading', 'CAS', 'TAS', 'Mach', 'SAT', 'Baro_Alt', 'Roll_Rate', 'Pitch_Rate', 'Heading_Rate', 'Fuel_Qty1', 'Fuel_Qty2', 'L_Eng_Start', 'R_Eng_Start', 'L_Throttle_Pos', 'R_Throttle_Pos', 'L_Eng_N1', 'R_Eng_N1', 'L_Eng_N2', 'R_Eng_N2', 'L_Gear_Down', 'R_Gear_Down', 'N_Gear_Down', 'L_Flaperon_Pos', 'R_Flaperon_Pos', 'L_LEF_Pos', 'R_LEF_Pos', 'L_Rudder_Pos', 'L_Stab_Pos', 'R_Stab_Pos', 'Stick_Pitch', 'Stick_Roll', 'Pedal_Pos']
Targets : ['Strain1', 'Strain2', 'Strain3', 'Strain4', 'Strain5', 'Strain6', 'Strain7', 'Strain8', 'Strain9', 'Strain10', 'Strain11', 'Strain12', 'Strain13', 'Strain14', 'Strain15']


In [7]:
# Remove linhas com NaN nas colunas importantes
df_clean = df.dropna(subset=feature_cols + target_cols)

X = df_clean[feature_cols].values      # entradas
y = df_clean[target_cols].values      # saídas (strains)
Tol_ids = df_clean["Tol_ID"].values  # de qual TOL é cada linha

In [8]:

X_train, X_temp, y_train, y_temp, ids_train, ids_temp = train_test_split(
    X, y, Tol_ids,
    test_size=0.3,
    random_state=42,
    stratify=Tol_ids
)

X_val, X_test, y_val, y_test, ids_val, ids_test = train_test_split(
    X_temp, y_temp, ids_temp,
    test_size=0.5,
    random_state=42,
    stratify=ids_temp
)

In [9]:
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X_train_scaled = scaler_X.fit_transform(X_train)
X_val_scaled   = scaler_X.transform(X_val)
X_test_scaled  = scaler_X.transform(X_test)

y_train_scaled = scaler_y.fit_transform(y_train)
y_val_scaled   = scaler_y.transform(y_val)
y_test_scaled  = scaler_y.transform(y_test)

In [38]:
class StrainDataset(Dataset):
    def __init__(self, X, y, seq_len):
        self.X = X
        self.y = y
        self.seq_len = seq_len

    def __len__(self):
        return len(self.X) - self.seq_len + 1

    def __getitem__(self, idx):
        x_seq = self.X[idx : idx + self.seq_len]
        y_seq = self.y[idx : idx + self.seq_len]
        return torch.tensor(x_seq, dtype=torch.float32), torch.tensor(y_seq, dtype=torch.float32)
sequence_len1 = 20
sequence_len2 = 100
train_ds = StrainDataset(X_train_scaled, y_train_scaled, sequence_len2)
val_ds   = StrainDataset(X_val_scaled,   y_val_scaled, sequence_len2)
test_ds  = StrainDataset(X_test_scaled,  y_test_scaled, sequence_len2)

train_loader = DataLoader(train_ds, batch_size=256, shuffle=True)
val_loader   = DataLoader(val_ds,   batch_size=256, shuffle=False)
test_loader  = DataLoader(test_ds,  batch_size=256, shuffle=False)

Notre problème est dynamique car différentes variables dépendent du temps. Comme les variables sont nombreuses il serait aussi intéressant de réduire nos nombres de variables pour caractériser au mieux dans un minimum de variable notre problème. Ainsi, la solution choisit serait un `Multi-modèle` :
- En premier lieu, nous allons réduire nos variables en un espace latent z avec un `Encodeur` qui caractérise au mieux "spatialement" notre problème.
- Nous souhaitons aussi avoir des "fenêtres temporelles d'informations" afin de relier certaine grandeur physiques au temps. Ainsi, la seconde "couche" de notre modèle seront des couches `LSTM` qui relie nos informations au temps, et qui rend un tenseur avec les derniers état caché de chaque cellule.
- Finalement, nos informations passe par des dernières couches d'un `MLP` linéaire standard qui permettent la prédiction des contraintes appliquée à notre avion.

***Encoder layers***

# Optimizing hyperparameters


à l'inverse du premier modèle instancié au hasard, on propose ici un mlti modèle "variable"(avec des valeurs par défaut si non précisées). on pourra part la suite choisir des intervalles de recherches pour nos hyper paramètre avec un optimiseur d'hyperparamètre. Celui choisit pour l'étude sera `Optuna`. 

In [ ]:
class Encoder(nn.Module):#réduction de nos variable en un espace latent z qui caractérise au mieux nos données.
    def __init__(self, input_size, nb_percp=64, nb_layers=3, z_dim = 16):
        super().__init__() #on appelle la classe parent Module
        layers = []
        in_dim = input_size
        for _ in range(nb_layers): #comme le nombre de couches est inconnu, il est donc nécessaire d'utiliser une boucle for pour créer notre classe
            linear = nn.Linear(in_dim, nb_percp)
            kaiming_uniform_(linear.weight, nonlinearity='relu')
            layers.append(linear)#on ajoute la couche linéaire à une liste qui correspond à notre réseau caché - la dernière couche
            layers.append(nn.ReLU()) #sans oublier les fonction d'activation
            in_dim = nb_percp # après être passé une première fois dans la boucle, nos couches auront le même nombres de neuronnes
        output = nn.Linear(in_dim, z_dim) 
        kaiming_uniform_(output.weight, nonlinearity='relu')
        layers.append(output) #on ajoute la dernière couche de notre encoder
        self.net = nn.Sequential(*layers) #on appelle le pointeur de la liste pour récupérer les différentes couches et créer le "net" total.
         #la fonction reLU a le gradiant le plus stable, elle semble donc être un bon choix pour notre projet.
    def forward (self, x): #passage du tenseur dans les différentes couches de notre Encoder
        return self.net(x)
        

***LSTM Layers***

In [40]:
class Lstm(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, nb_layers: int, dropout: float = 0.1):
        """
            on initialise notre système avec un dropout faible
        """
        super().__init__()
        self.input_size = input_size # récupère la taille des vecteurs des valeurs d'entrées
        self.hidden_size = hidden_size  # récupère le nombre de cellules par couche de lstm
        self.nb_layers = nb_layers  # récupère le nombre de couche dans le LSTM
        # Défini les couches LSTM
        self.lstm = torch.nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=nb_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        # on fait passer l'entrée par les différente cellules du lstm
        output, (h_n, c_n) = self.lstm(x)
        return output
    

***MLP predictive layers***

In [ ]:
class MLP_v2(nn.Module):
    def __init__(self, input_size, nb_percp = 128, nb_layers = 3, n_contraintes = 15):
        super().__init__()
        layers = []
        in_dim = input_size
        for _ in range(nb_layers):#on applique la même réflexion que l'encoder au mlp qui donne les différentes contraintes en sortie.
            linear = nn.Linear(in_dim, nb_percp)
            kaiming_uniform_(linear.weight, nonlinearity='relu')
            layers.append(linear)
            layers.append(nn.ReLU())
            in_dim = nb_percp
        output = nn.Linear(in_dim, n_contraintes)
        kaiming_uniform_(output.weight, nonlinearity='linear')
        layers.append(output)
        self.net = nn.Sequential(*layers)
         #la fonction reLU a le gradiant le plus stable, elle semble donc être un bon choix pour notre projet.
    def forward (self, x): #passage du tenseur dans les différentes couches de notre Encoder.
        return self.net(x)

**Multi-modèle**

In [42]:
class Mercosur(nn.Module):
    def __init__(self, input_size, nb_percp_enc, nb_layers_enc, z_dim, hidden_size_lstm, nb_layers_lstm, nb_percp_mlp, nb_layers_mlp, n_contraintes):
        super().__init__()
        self.encoder_layers = Encoder(input_size, nb_percp_enc, nb_layers_enc, z_dim)
        self.lstm_layers = Lstm(z_dim, hidden_size_lstm, nb_layers_lstm)
        # self.mlp_layers = MLP(hidden_size, n_contraintes).
        self.mlp_layers = MLP_v2(hidden_size_lstm, nb_percp_mlp, nb_layers_mlp, n_contraintes)


    def forward(self, x):
        # x: (batch, seq_len, n_features)
        z_seq = self.encoder_layers(x)      # (batch, seq_len, z_dim)
        h_seq = self.lstm_layers(z_seq)     # (batch, seq_len, hidden_size)
        y_pred = self.mlp_layers(h_seq)     # (batch, seq_len, n_contraintes)
        return y_pred

In [43]:
def train_epoch(model, loader, optimizer, criterion):
    model.train()
    running_loss = 0.0
    for i, (X_batch, y_batch) in enumerate(tqdm(loader)):
        # ver o shape só no primeiro batch
        if i == 0:
            print("X_batch shape:", X_batch.shape)
            print("y_batch shape:", y_batch.shape)

        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)
        optimizer.zero_grad()
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)

    return running_loss / len(loader.dataset)

def eval_epoch(model, loader, criterion):
    model.eval()
    running_loss = 0.0
    with torch.no_grad():
        for i, (X_batch, y_batch) in enumerate(loader):

            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            running_loss += loss.item() * X_batch.size(0)

    return running_loss / len(loader.dataset)


In [44]:
print({next(iter(train_loader))[0].shape})

{torch.Size([256, 100, 39])}


In [45]:
import torch
def train(capteur_model, train_loader, val_loader, optimizer, criterion, epochs = 30):
    train_losses, val_losses = [], []
    for epoch in range(epochs):
        train_loss = train_epoch(capteur_model, train_loader, optimizer, criterion)
        val_loss   = eval_epoch(capteur_model, val_loader, criterion)
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        print(f"Epoch {epoch+1:03d} | Train: {train_loss:.6f} | Val: {val_loss:.6f}")
    return capteur_model, train_losses, val_losses


## Optimisation des hyperparamètres

Le multi modèle devrai avoir une représentation spatiale et temporelle "juste" du problème. par rapport au modèle précédent, la séquence d'étude du LSTM a augmentée. Nous sommes passé de 20 pas de temps à 100. Certain phénomène physique récupérés par les différents capteurs sont reliée au temps mais si l'intervalle étudiée est trop petite, il se pourrait que seulement un régime transitoire soit étudié (pas de temps de 0.032s). il reste à déterminer à quel point nos hyperparamètres sont fiable. Afin de s'en assurer, on utilise la librairie optuna, un optimiseur d'hyperparamètres.

In [46]:
import optuna
import torch.optim as optim

**pour l'instant, l'optimisation se fera sur plusieurs faibles couples d'hyperparamètres afin de ne pas trop surcharger le gpu et diminuer le temps de réponse.**

In [ ]:


#on récupère la taille du vecteur d'entrée et de sortie voulu pour notre multi-modèle
n_features = X_train_scaled.shape[-1]
n_outputs  = y_train_scaled.shape[-1]

On instancie une fonction objective1 qui donne en sortie la métrique que l'on souhaite minimiser avec l'optimiseur d'hyper paramètres

In [ ]:


def objective1(trial):
    # nb_percps_enc = trial.suggest_int("nb_percps_enc",30, 60)
    # nb_percps_mlp = trial.suggest_int("nb_percps_mlp",128, 256)
    # num_layersenc = trial.suggest_int("num_layers_enc", 1, 5)
    # num_layerslstm = trial.suggest_int("num_layers", 1, 5)
    # num_layersmlp = trial.suggest_int("num_layers", 1, 5)
    n_features = X_train_scaled.shape[-1]
    n_outputs  = y_train_scaled.shape[-1]
    #ici on fixe des valeurs afin d'assurer un temps de réponse "plutot" faible. Cela serait trop couteûx de minimiser la rmse avec 8variables à déterminer
    nb_percps_enc = 32 
    nb_percps_mlp = 128
    num_layersenc = 3
    num_layerslstm = 3
    num_layersmlp = 3
    #nous porterons notre attention dans un premier temps sur z_dim et le nombre de cellules du lstm
    #afin de savoir quelle sont les meilleure valeurs pour caractériser spatialement et temporellement notre problème
    z_dim = trial.suggest_int("z_dim", 10, 20)
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    hidden_size_lstm = trial.suggest_int("hidden_size", 30, 70)
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    opti_mercosur = Mercosur(n_features, nb_percps_enc, num_layersenc, z_dim, hidden_size_lstm, num_layerslstm, nb_percps_mlp, num_layersmlp, n_outputs).to(device)
    loss_fn = nn.MSELoss()
    opti_optimizer = optim.Adam(opti_mercosur.parameters(), lr= lr)
    train(opti_mercosur, train_loader, val_loader, opti_optimizer, loss_fn)

    opti_mercosur.eval()
    y_pred_list = []
    
    with torch.no_grad():
        for X_batch in test_loader:
            X_batch = X_batch.to(device)
            y_pred = opti_mercosur(X_batch)
            y_pred_list.append(y_pred.cpu().numpy())
    y_pred_scaled = np.concatenate(y_pred_list, axis=0)
    y_pred_scaled_2d = y_pred_scaled.reshape(-1, D)  # (N*T, D)
    y_pred_real = scaler_y.inverse_transform(y_pred_scaled_2d)
    rmse_global = mean_squared_error(y_test, y_pred_real, squared=False)
    return rmse_global #on choisira la rmse à minimiser car c'est celle demandée par le projet


on créer une étude Optuna qui récupère la fonction objective1 et qui avec n_trials essai, essaie de minimiser la rmse( et plus généralement la sortie de notre fonction prit en argument).

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective1, n_trials = 30) #ili y aura 30 essai pour minimiser la rmse
print("Best params:", study.best_params)
print("Best MSE:", study.best_value)

## Validation

on récupère le modèle avec la meilleure rmse.

In [ ]:
nb_percps_enc = 32 
nb_percps_mlp = 128
num_layersenc = 3
num_layerslstm = 3
num_layersmlp = 3
n_features = X_train_scaled.shape[-1]
n_outputs  = y_train_scaled.shape[-1]
best_model = study.best_trial
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
best_mercosur = Mercosur(n_features, nb_percps_enc, num_layersenc, best_model["z_dim"], best_model["hidden_size_lstm"], num_layerslstm, nb_percps_mlp, num_layersmlp, n_outputs).to(device)
loss_fn = nn.MSELoss()
best_optimizer = optim.Adam(best_mercosur.parameters(), lr= best_model["lr"])
train_losses, test_losses, final_mercosur = train(best_mercosur, train_loader, val_loader, best_optimizer, loss_fn)

IndentationError: unexpected indent (597658585.py, line 3)

In [ ]:
torch.save(final_mercosur.state_dict(), "best_mercosur.pth")
print("model saved in 'best_mercosur.pth'")

In [ ]:

plt.figure(figsize=(10,6))

epochs = range(1, len(train_losses) + 1)

plt.plot(epochs, train_losses, label="Train Loss", marker='o')
plt.plot(epochs, val_losses,   label="Validation Loss", marker='s')

plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.title("Train vs Validation Loss")
plt.grid(True)
plt.legend()
plt.savefig('Train_vs_Validation_Loss_Mercosur.png', dpi=300)
plt.tight_layout()
plt.show()


In [ ]:
img = mpimg.imread("Train_vs_Validation_Loss_Mercosur.png")

plt.figure(figsize=(12, 10))
plt.imshow(img)
plt.axis("off")
plt.show()

In [ ]:
final_mercosur.eval()


y_pred_list = []

with torch.no_grad():
    for X_batch, y_batch in test_loader:
        X_batch = X_batch.to(device)

        # forward no modelo
        y_pred = final_mercosur(X_batch)  # (batch, seq_len, n_outputs)

        # manda tudo pra CPU e acumula
        
        y_pred_list.append(y_pred.cpu().numpy())

y_pred_scaled = np.concatenate(y_pred_list, axis=0)  # (N, T, D)

# achata a dimensão temporal pra usar no scaler (espera 2D)
N, T, D = y_pred_scaled.shape

y_pred_scaled_2d = y_pred_scaled.reshape(-1, D)  # (N*T, D)

# volta pro espaço real

y_pred_real = scaler_y.inverse_transform(y_pred_scaled_2d)

# métricas globais
mae_global  = mean_absolute_error(y_test, y_pred_real)
rmse_global = mean_squared_error(y_test, y_pred_real, squared=False)
r2_global   = r2_score(y_test, y_pred_real)

print("Overall metrics in the test set:")
print(f"MAE  : {mae_global:.4f}")
print(f"RMSE : {rmse_global:.4f}")
print(f"R²   : {r2_global:.4f}")

Overall metrics in the test set:
MAE  : 20.5281
RMSE : 34.9109
R²   : 0.9500


lire les images de le training up side

In [ ]:
T = y_pred_scaled.shape[1]
N_seq_pred = y_pred_scaled.shape[0]

ids_test_eff = ids_test[:N_seq_pred]
ids_test_step = np.repeat(ids_test_eff, T)

metrics_per_flight = {}
unique_flights = np.unique(ids_test_eff)

for flight in unique_flights:
    mask = (ids_test_step == flight)

    y_true_f = y_test[mask]
    y_pred_f = y_pred_real[mask]

    mae  = mean_absolute_error(y_true_f, y_pred_f)
    rmse = mean_squared_error(y_true_f, y_pred_f, squared=False)
    r2   = r2_score(y_true_f, y_pred_f)

    metrics_per_flight[flight] = {"MAE": mae, "RMSE": rmse, "R2": r2}


In [ ]:
df_flights = pd.DataFrame.from_dict(metrics_per_flight, orient='index')

# adicionar as métricas globais
df_global = pd.DataFrame({
    "MAE":  [mae_global],
    "RMSE": [rmse_global],
    "R2":   [r2_global]
}, index=["GLOBAL"])

# concatenar tudo
df_results = pd.concat([df_flights, df_global])



In [ ]:
df_results.to_csv('df_results_multi_model.csv')

In [ ]:
# df_results = pd.read_csv('df_results_all.csv')
df_results


,MAE,RMSE,R2
TOL_1,21.411116,35.396049,0.950829
TOL_10,21.469473,35.473278,0.949585
TOL_11,21.668261,35.956020,0.948748
TOL_12,21.871603,36.244915,0.948428
TOL_13,21.405066,35.142048,0.950851
TOL_14,21.944002,36.332851,0.948736
TOL_15,21.899666,36.122211,0.949418
TOL_17,21.490608,35.413269,0.950260
TOL_18,21.938776,36.316383,0.948756
TOL_19,21.660200,36.023014,0.948947
